In [2]:
import numpy as np
import pandas as pd

from vajeED_1_podatki import *

# Odkrivanje enačb, vaje 1

# 1. del: linearna regresija

Ena najpreprostejših in pogosto precej učinkovitih metod za odkrivanje enačb je redka linearna regresija. Omejena je na enačbe, linearne v parametrih, omogoča nam pa dodajanje členov s poljubnimi oblikami funkcij. Pri linearni regresiji iščemo model oblike $\hat{y}_i = \sum\limits_{k=0}^K \beta_k X_{ik}$, kjer je $\vec{\beta}$ vektor koeficientov, $X$ pa matrika podatkov. Če minimiziramo kvadratno napako $\sum_i (\hat{y}_i - y_i)^2$, se da izpeljati analitično rešitev za koeficiente:

$$\vec{\beta} = (X^TX)^{-1}X^T \vec{y}$$

Pri uporabi linearne regresije za odkrivanje enačb so ključne dodatne spremenljivke, ki jih zgeneriramo. To so lahko višji redi spremenljivk, produkti spremenljivk, logaritmi, trigonometrične funkcije, itd. 

Za razliko od navadnega strojnega učenja nas pri odkrivanju enačb ne zanimajo preveč napovedi, temveč sam model - najboljša enačba, ki smo jo uspeli odkriti. Ker želimo čim bolj razumljive enačbe, je dobro iz končne enačbe odstraniti člene z zelo majhnimi koeficienti. Berljivost enačbe lahko dodatno izboljšamo tako, da v stringu vrednosti koeficientov zaokrožimo na manjše število decimalnih mest.

1.1 Napiši funkcijo za linearno regresijo. Sprejme naj:
- X: tabela z N vrsticami, ki ustrezajo N učnim primerom, ter K stolpci, ki ustrezajo K spremenljivkam (v splošnem bodo to originalne spremenljivke plus neko število dodatnih členov), imena stolpcev pa povejo, za kateri člen gre (np. "log(x-y)")
- y: vektor z N vrednostmi, ki ustrezajo levi strani enačbe.
- meja: opcijski argument, ki pove, pod katero vrednostjo koeficienta člen odstranimo iz enačbe.

Funkcija naj vrne:
- najdeno enačbo v obliki stringa (npr. "0.32 x + 0.10 x**2 - 2.02 sin(y)")


In [18]:
def to_signed_str(number):
    if number < 0:
        return f"{number:.4f}"
    else:
        return f"{number:+.4f}"
    
def linearna_regresija(X, y, meja=1e-2):
    names = list(X.columns.values)
    X = X.to_numpy()
    A = np.dot(X.transpose(), X)
    lhs = np.dot(X.transpose(), y)
    b = np.linalg.solve(A, lhs)
    b = map(to_signed_str, b)

    eq = np.array(list(zip(b, names))).flatten()
    return ''.join(eq)


1.2 Preizkusi svojo funkcijo za linearno regresijo na primeru odkrivanja energijskega zakona $E = mgh + \frac{1}{2}mv^2$. Za generiranje podatkov lahko uporabiš funkcijo **generiraj_energijski_zakon** iz datoteke **vajeED_1_podatki.py**. Podaš željeno število primerov (recimo 100) ter željeno stopnjo multiplikativnega šuma.

Zgenerirati bo potrebno tudi dodatne člene. To lahko sprogramiraš ročno, lahko pa uporabiš kakšno od funkcij iz scikit-learn. Za generiranje polinomskih členov je uporabna **sklearn.preprocessing.PolynomialFeatures**.

Kako dobro deluje navadna linearna regresija? Preizkusi vsaj tri nivoje šuma - smiselne vrednosti so recimo 0, 0.001, 0.01 in 0.1.

In [25]:
podatki_energija = generiraj_energijski_zakon(100, sum=0.)
def add_mult(table, a, b):
    table[a+b] = table[a].to_numpy()*table[b].to_numpy()
    return table

podatki_energija = add_mult(podatki_energija, "m", "h")
podatki_energija = add_mult(podatki_energija, "m", "v")
podatki_energija = add_mult(podatki_energija, "mv", "v")


linearna_regresija(podatki_energija.drop(["E"], axis=1), podatki_energija["E"])

'-0.0000m-0.0000h+0.0000v+9.8100mh+0.0000mv+0.5000mvv'

In [29]:
podatki_energija2 = generiraj_energijski_zakon(100, sum=0.001)
podatki_energija2 = add_mult(podatki_energija2, "m", "h")
podatki_energija2 = add_mult(podatki_energija2, "m", "v")
podatki_energija2 = add_mult(podatki_energija2, "mv", "v")


linearna_regresija(podatki_energija2.drop(["E"], axis=1), podatki_energija2["E"])

'-0.0052m+0.0012h-0.0012v+9.8122mh+0.0178mv+0.4878mvv'

1.3 Delovanje pri zašumljenih podatkih lahko včasih izboljšamo z regularizacijo. 
Pri redki (ang. sparse) regresiji želimo obržati vrednosti koeficientov nizke. Visoke vrednosti kaznujemo tako, da v funkcijo napake dodamo regularizacijski člen (ang. ridge regression):  $\sum_i (\hat{y}_i - y_i)^2 + \lambda \sum_k \beta_k^2$, kjer parameter $\lambda$ določa nivo regularizacije. Optimizacijski problem se da še vedno rešiti analitično:

$$ \vec{\beta} = (X^TX + \lambda I)^{-1}X^T \vec{y} $$ 

Napiši funkcijo za ridge regresijo. Sprejme naj:
- X: tabela z N vrsticami, ki ustrezajo N učnim primerom, ter K stolpci, ki ustrezajo K spremenljivkam (v splošnem bodo to originalne spremenljivke plus neko število dodatnih členov), imena stolpcev pa povejo, za kateri člen gre (np. "log(x-y)")
- y: vektor z N vrednostmi, ki ustrezajo levi strani enačbe.
- lambda: regularizacijski parameter, dobro vrednost za dani problem je treba določiti empirično (s poskušanjem)
- meja: opcijski argument, ki pove, pod katero vrednostjo koeficienta člen odstranimo iz enačbe


Funkcija naj vrne:
- najdeno enačbo v obliki stringa (npr. "0.32 x + 0.10 x**2 - 2.02 sin(y)")

Funkcijo za ridge regresijo preveri na podatkih za energijski zakon.

In [33]:
def ridge_regresija(X, y, lam=1, meja=1e-2):
    names = list(X.columns.values)
    X = X.to_numpy()
    A = np.dot(X.transpose(), X)
    A += lam*np.identity(len(A))
    lhs = np.dot(X.transpose(), y)
    b = np.linalg.solve(A, lhs)
    b = map(to_signed_str, b)

    eq = np.array(list(zip(b, names))).flatten()
    return ''.join(eq)


In [35]:
ridge_regresija(podatki_energija2.drop(["E"], axis=1), podatki_energija2["E"], lam=0.001)

'-0.0032m+0.0072h-0.0063v+9.8001mh+0.0265mv+0.4862mvv'

1.4 Ridge regresija za ta problem očitno ne deluje najboljše. V resnici je za odkrivnaje enačb bolj primerna regularizacija z normo L1:
$\sum_i (\hat{y}_i - y_i)^2 + \lambda \sum_k |\beta_k|$.
Taki linearni regresiji rečemo Lasso in žal nima analitične rešitve, zato moramo optimalne vrednosti parametrov $\beta_k$ iskati z numerično minimizacijo (recimo **scipy.optimize.minimize**).

Napiši funkijo za Lasso regresijo z istimi vhodi in izhodi kot v 1.3 ter jo preizkusi na podatkih za energijski zakon. Ali deluje bo


In [62]:
from scipy.optimize import minimize

def f(X, y, lam, x):
    inner = np.dot(X, x) - y
    v = inner * inner
    return np.linalg.norm(v, 1) + lam* np.linalg.norm(x, 1)

def lasso_regresija(X, y, lam=1, meja=1e-2):
    names = list(X.columns.values)
    X = X.to_numpy()
    y = y.to_numpy()
    partial = lambda x : f(X, y, lam, x)
    x0 = np.zeros(np.shape(X)[1])
    b = minimize(partial, x0)
    b = b["x"]
    b = map(to_signed_str, b)

    eq = np.array(list(zip(b, names))).flatten()
    return ''.join(eq)

In [66]:
lasso_regresija(podatki_energija2.drop(["E"], axis=1), podatki_energija2["E"], lam=0.1)

'-0.0000m+0.0021h+0.0021v+9.8029mh+0.0260mv+0.4629mvv'

DODATNO: Odkrivanje enačb z linearno regresijo lahko preizkusiš tudi na drugih podatkih v datoteki **vajeED_1_podatki.py**.

DODATNO-2: Na vajah smo spoznali osnovno implementacijo redke regresije za odkrivanje enačb. Za bolj resne implementacijo si lahko pogledaš knjižnico SINDy, ki je namenjena predvsem odkrivanju sistemov diferencialnih enačb, ki opisujejo dinamične sisteme: https://github.com/dynamicslab/pysindy.

## 2. del - BACON
Na predavanjih smo spoznali enega prvih algoritmov za odkrivanje enačb - BACON. Ta algoritem je koristen za odkrivanje enačb, ki vključujejo samo množenje in deljenje. 

![algoritem BACON](bacon.png)

Zapiši funkcijo, ki sprejme tabelo T s stolpci $x_i$, $1 \le i \le m$ in najde zvezo $c = E(x_1, \dots , x_m)$ po Baconovi metodi (alg. 1). Premisli, kako poimenovati nove spremenljivke in ali se da pogoja iz druge in tretje veje if-stavka malo razrahljati.

Metodo preizkusi na Newtonovem zakonu $F = ma$ (**generiraj_newton**) in Stefanovem zakonu $j = \sigma T^4$ (**generiraj_stefan**).

In [ ]:
def bacon(df, max_iter=20):
    stolpci = list(df.columns)
    for i in range(max_iter):
        df_array = np.array(df)
        """ocenimo, ali obstaja konstanta"""
        sig = np.std(df_array, axis=0)
        # DOPOLNI
        """izracunajmo vrstne rede, najdemo korelacije med njimi"""
        vrstni_redi = df.rank(axis=0)
        korelacije = np.corrcoef(df_array, df_array)
        # DOPOLNI
        """oglej si np.max(korelacije) in np.min(korelacije)"""
        # DOPOLNI
        """pametno poimenuj novi stolpec, da bomo lahko iz njega razbrali enacbo"""
        # DOPOLNI





In [ ]:
podatki = generiraj_newton(100, sum=0)
bacon(podatki)

DODATNO: Če te odkrivanje enačb zanima, si lahko pogledaš še modernejše algoritme. 

**pySR** je enostavna za instalacijo, temelji pa na genetskih algoritmih: https://github.com/MilesCranmer/PySR

**DSO** je trenutno ena od najmočnejših metod za odkrivanje enačb, združuje pa globoko učenje ter genetske algoritme: https://github.com/brendenpetersen/deep-symbolic-optimization